# PSI运行示例

初始化mpc设备

In [1]:
from common import MPCInitializer
import secretflow as sf
mpc_init = MPCInitializer()
company, partner = mpc_init.company,mpc_init.partner
heu_devices = (mpc_init.company_heu, mpc_init.partner_heu)

/home/lvx_vitae/anaconda3/envs/sf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-17 15:07:21,341	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-09-17 15:07:21,648	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/lvx_vitae/anaconda3/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
/home/lvx_vitae/anaconda3/envs/sf/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompa

(SPURuntime pid=382308) 2025-09-17 15:07:27.882 [warning] [openssl_factory.cc:OpensslDrbg:83] Yacl has been configured to use Yacl's entropy source, but unable to find one. Fallback to use openssl's default entropy srouce
(SPURuntime pid=382308) 2025-09-17 15:07:27.890 [warning] [openssl_factory.cc:OpensslDrbg:83] Yacl has been configured to use Yacl's entropy source, but unable to find one. Fallback to use openssl's default entropy srouce


(HEUSkKeeper(heu_id=132287314964304, party=company) pid=382312) [2025-09-17 15:07:34.061] [info] [thread_pool.cc:30] Create a fixed thread pool with size 15
(HEUEvaluator(heu_id=132287284148784, party=company) pid=382314) [2025-09-17 15:07:33.726] [info] [thread_pool.cc:30] Create a fixed thread pool with size 15


以下示例仅用于说明PSI的工作流程，目前仅支持单机测试。

双方读取数据集。应将键值和特征分开读取。

In [2]:
import pandas as pd
import os
import numpy as np
project_dir = os.path.dirname(os.path.abspath(''))
data_dir = os.path.join(project_dir, 'Datasets', 'PSI')

def read_dataset(path: str):
    data = pd.read_csv(path)
    keys = data.iloc[:, 0].tolist()
    private_features = data.iloc[:, 1:].to_numpy(dtype=np.float32)
    return (keys, private_features,np.arange(len(keys)))
company_data = company(read_dataset)(os.path.join(data_dir, 'company_data.csv'))
partner_data = partner(read_dataset)(os.path.join(data_dir, 'partner_data.csv'))

执行PSI

求交后得到交集共享分片，分别由company和partner持有。原本company数据的特征在share的左侧，partner数据的特征在右侧。

In [3]:
from PSI import private_set_intersection
company_share, partner_share, bucket_labels = private_set_intersection(company_data, partner_data,heu_devices)

Computing masked company cipher
Computing masked partner cipher
Computing company shares
Computing partner shares


结果比较

In [4]:
real_intersection = pd.read_csv(os.path.join(data_dir, 'intersection.csv'))
real_intersection = real_intersection.iloc[:, 1:].to_numpy()
print("Real Intersection:", real_intersection)

company_share = sf.reveal(company_share)
partner_share = sf.reveal(partner_share)
print("PSI intersection:",company_share + partner_share)
print("Bucket labels:", bucket_labels)

Real Intersection: [[96.94997288 94.94783572 45.7531272  91.97591751  6.40621559]
 [49.26746318 41.47538697 14.44682987 33.89487272 99.77746512]
 [97.20440369 85.14677116 76.61730259 74.54188118 91.99566983]
 [99.33640613 60.24766471 75.91638368 10.36925581 66.76225286]
 [99.5435931  14.6643015  92.38456288 28.50034193 94.4518885 ]]
PSI intersection: [[99.33640289 60.24766541 12.         75.91638184 10.36925602 66.76225281
   9.        ]
 [96.94997406 94.94783783  2.         45.75312805 91.975914    6.40621567
   3.        ]
 [99.54359436 14.66430187 14.         92.38455963 28.50034142 94.45188904
   2.        ]
 [97.20440674 85.14677429  6.         76.61730194 74.54187775 91.9956665
   4.        ]
 [49.26746368 41.47538757  3.         14.4468298  33.89487457 99.77746582
   8.        ]]
Bucket labels: [12  2 14  6  3  9  3  2  4  8]
